# Домашнее задание к лекции "Базовые понятия статистики"

Будем осуществлять работу с непростым [набором данных](https://raw.githubusercontent.com/obulygin/pyda_homeworks/master/statistics_basics/horse_data.csv) о состоянии здоровья лошадей, испытывающих кишечные колики. Цель – максимально корректно заполнить пропуски.

### Задание 1. Загрузка данных

Изучить представленный набор данных на основе [описания его столбцов](https://raw.githubusercontent.com/obulygin/pyda_homeworks/master/statistics_basics/horse_data.names), загрузить его и оставить 8 столбцов для дальнейшего изучения: `surgery?`, `Age`, `rectal temperature`, `pulse`, `respiratory rate`, `temperature of extremities`, `pain`, `outcome`. 

In [2]:
import pandas as pd
import numpy as np

In [3]:
header = ['surgery?', 'age', 'Hospital Number', 'rectal temperature', 'pulse', 'respiratory rate',\
          'temperature of extremities', 'peripheral pulse', 'mucous membranes', 'capillary refill time',\
          'pain', 'peristalsis', 'abdominal distension', 'nasogastric tube', 'nasogastric reflux',\
          'nasogastric reflux PH', 'rectal examination - feces', 'abdomen', 'packed cell volume',\
          'total protein', 'abdominocentesis appearance', 'abdomcentesis total protein', 'outcome',\
          'surgical lesion?', 'site of lesion', 'type', 'specific code', 'cp_data']

header_needed = ['surgery?', 'age', 'rectal temperature', 'pulse', 'respiratory rate',\
                 'temperature of extremities', 'pain', 'outcome']

df = pd.read_csv('https://raw.githubusercontent.com/obulygin/pyda_homeworks/master/statistics_basics/horse_data.csv',\
                 names = header, na_values = '?')

df = df[header_needed]

In [4]:
df.head()

,surgery?,age,rectal temperature,pulse,respiratory rate,temperature of extremities,pain,outcome
0,2.0,1,38.5,66.0,28.0,3.0,5.0,2.0
1,1.0,1,39.2,88.0,20.0,NaN,3.0,3.0
2,2.0,1,38.3,40.0,24.0,1.0,3.0,1.0
3,1.0,9,39.1,164.0,84.0,4.0,2.0,2.0
4,2.0,1,37.3,104.0,35.0,NaN,NaN,2.0


### Задание 2. Первичное изучение данных

Проанализировать значения по столбцам, рассчитать базовые статистики, найти выбросы.

In [5]:
df.describe() #основные характеристики разом

,surgery?,age,rectal temperature,pulse,respiratory rate,temperature of extremities,pain,outcome
count,299.000000,300.000000,240.000000,276.000000,242.000000,244.000000,245.00000,299.000000
mean,1.397993,1.640000,38.167917,71.913043,30.417355,2.348361,2.95102,1.551839
std,0.490305,2.173972,0.732289,28.630557,17.642231,1.045054,1.30794,0.737187
min,1.000000,1.000000,35.400000,30.000000,8.000000,1.000000,1.00000,1.000000
25%,1.000000,1.000000,37.800000,48.000000,18.500000,1.000000,2.00000,1.000000
50%,1.000000,1.000000,38.200000,64.000000,24.500000,3.000000,3.00000,1.000000
75%,2.000000,1.000000,38.500000,88.000000,36.000000,3.000000,4.00000,2.000000
max,2.000000,9.000000,40.800000,184.000000,96.000000,4.000000,5.00000,3.000000


In [6]:
# моды

print('surgery? - ', df['surgery?'].mode()[0])
print('age - ', df['age'].mode()[0])
print('rectal temperature - ', df['rectal temperature'].round().mode()[0])
print('pulse - ', df['pulse'].round().mode()[0])
print('respiratory rate - ', df['respiratory rate'].round().mode()[0])
print('temperature of extremities - ', df['temperature of extremities'].mode()[0])
print('pain - ', df['pain'].mode()[0])
print('outcome - ', df['outcome'].mode()[0])

surgery? -  1.0
age -  1
rectal temperature -  38.0
pulse -  48.0
respiratory rate -  20.0
temperature of extremities -  3.0
pain -  3.0
outcome -  1.0


In [7]:
# СКО
for i in df:
    print(i, ' -- ', np.std(df[i], ddof=1))

surgery?  --  0.49030464432881815
age  --  2.1739719055576163
rectal temperature  --  0.7322886641121578
pulse  --  28.630556660735003
respiratory rate  --  17.642231385134664
temperature of extremities  --  1.0450536920112758
pain  --  1.3079395711243502
outcome  --  0.7371869448054933


In [8]:
# Дисперсия

for i in df:
    print(i, ' -- ', df[i].var())

surgery?  --  0.24039864425040888
age  --  4.726153846153814
rectal temperature  --  0.5362466875871686
pulse  --  819.7087747035575
respiratory rate  --  311.24832824663054
temperature of extremities  --  1.0921372191863987
pain  --  1.710705921712949
outcome  --  0.5434445915916574


In [9]:
def outlier_search(df, col, how_many = 2):
    '''
    Функция ищет в датафрейме df выбросы по одной из колонок col и возвращает список индексов строк, в которых эти выбросы находятся.
    Параметр how_many определяет:
    0 - вывести только выбросы за нижнюю границу
    1 - вывести только выбросы за верхнюю границу
    2 - вывести выбросы за обеими границами (по умолчанию)
    '''
    q1 = df[col].quantile(0.25) # первый квартиль
    q3 = df[col].quantile(0.75) # третий квартиль
    iqr = q3 - q1 # межквартильный размах
    lower_bound = q1 - (1.5 * iqr) # нижняя граница выбросов
    upper_bound = q3 + (1.5 * iqr) # верхняя граница выбросов
    if how_many == 2:
        return list(df[(df[col]<lower_bound) | (df[col]>upper_bound)].index)
    elif how_many == 1:
        return list(df[df[col]>upper_bound].index)
    else:
        return list(df[col]<lower_bound)
    

In [10]:
df.iloc[outlier_search(df, 'pulse', 2), :]

,surgery?,age,rectal temperature,pulse,respiratory rate,temperature of extremities,pain,outcome
3,1.0,9,39.1,164.0,84.0,4.0,2.0,2.0
41,2.0,9,39.0,150.0,72.0,NaN,NaN,1.0
55,1.0,9,38.6,160.0,20.0,3.0,3.0,2.0
255,1.0,9,38.8,184.0,84.0,1.0,4.0,2.0
275,1.0,9,38.8,150.0,50.0,1.0,5.0,2.0


### Задание 3. Работа с пропусками

Рассчитать количество пропусков для всех выбранных столбцов. Принять и обосновать решение о методе заполнения пропусков по каждому столбцу на основе рассчитанных статистик и возможной взаимосвязи значений в них. Сформировать датафрейм, в котором пропуски будут отсутствовать.

In [11]:
# Рассчитаем количество пропусков:
# суммируем таблицу после обработки isna для получения количества пропусков
# в каждом столбце без написания дополнительных функций
df.isna().aggregate('sum')

surgery?                       1
age                            0
rectal temperature            60
pulse                         24
respiratory rate              58
temperature of extremities    56
pain                          55
outcome                        1
dtype: int64

In [12]:
# определим долю пропусков в данных по столбцам:
(df.isna().mean() * 100).round(2)

surgery?                       0.33
age                            0.00
rectal temperature            20.00
pulse                          8.00
respiratory rate              19.33
temperature of extremities    18.67
pain                          18.33
outcome                        0.33
dtype: float64

### Обоснование решения о методе заполнения пропусков
**1. "surgery?" и "outcome"**

    Строки с единственным пропуском можно отбросить.

In [27]:
df_clean = df.iloc[:,:] # создадим отдельный датафрейм, и он будет по окончании работы без пропусков.
df_clean = df_clean[-df_clean['surgery?'].isna()]
df_clean = df_clean[-df_clean['outcome'].isna()]

**2. "rectal temperature"**

    Группировка по различным полям медианой показывает, что заметной связи между полем и другими полями нет. Предлагаю заполнить медианой, сгруппированной по полям outcome и age, т.к. повышенная температура - индикатор проблем в организме, и чем старше организм, тем тяжелее переносить недуг. Примерно это прослеживается при группировке по полям outcome и age.

In [28]:
df_clean[['outcome', 'age', 'rectal temperature']].groupby(['outcome', 'age']).median()

rectal temperature
outcome age                    
1.0     1                  38.2
        9                  38.4
2.0     1                  38.0
        9                  38.8
3.0     1                  38.0
        9                  39.7

In [29]:
df_clean['rectal temperature'].fillna(df_clean.groupby(['outcome', 'age'])\
                                      ['rectal temperature'].transform('median'), inplace=True)

**3. "pulse"**
    
    Аналогично ректальной температуре, заметной связи с большинством полей нет. В связи с этим предлагаю определять пропуски медианой, сгруппированной по outcome и age, т.к. в целом есть тенденция к росту пульса с возрастом. А исход (outcome) тоже может влиять на частоту пульса.

In [30]:
df_clean[['outcome', 'age', 'pulse']].groupby(['outcome', 'age']).median()

pulse
outcome age       
1.0     1     52.0
        9    120.0
2.0     1     82.0
        9    141.0
3.0     1     81.0
        9    100.0

In [31]:
df_clean['pulse'].fillna(df_clean.groupby(['outcome', 'age'])\
                                      ['pulse'].transform('median'), inplace=True)

**4. "respiratory rate"**
    
    Предполагаем, что зависимость от возраста аналогична предыдущей переменной. Однако из группировки придётся убрать outcome, т.к. пропуски характерны именно для outcome = 3

In [32]:
df_clean[['age', 'respiratory rate']].groupby(['age']).median()

,respiratory rate
age,
1,24.0
9,49.0


In [33]:
df_clean['respiratory rate'].fillna(df_clean.groupby(['age'])\
                                      ['respiratory rate'].transform('median'), inplace=True)

**5. "temperature of extremities"**

    Заполняем исходя из предположения, что частота дыхания и outcome повлияют на температуру конечностей. Оставшиеся после этого NaN дозаполним на основании outcome.

In [34]:
df_clean[['respiratory rate', 'outcome', 'temperature of extremities']].groupby(['respiratory rate', 'outcome']).median()

temperature of extremities
respiratory rate outcome                            
8.0              1.0                             NaN
9.0              1.0                             2.0
10.0             1.0                             3.0
12.0             1.0                             1.5
                 2.0                             2.0
...                                              ...
84.0             2.0                             2.5
88.0             1.0                             1.0
90.0             1.0                             2.0
96.0             1.0                             NaN
                 2.0                             NaN

[72 rows x 1 columns]

In [37]:
df_clean[['outcome', 'temperature of extremities']].groupby(['outcome']).median()

,temperature of extremities
outcome,
1.0,2.0
2.0,3.0
3.0,3.0


In [35]:
df_clean['temperature of extremities'].fillna(df_clean.groupby(['respiratory rate', 'outcome'])\
                                      ['temperature of extremities'].transform('median'), inplace=True)

In [38]:
df_clean['temperature of extremities'].fillna(df_clean.groupby(['outcome'])\
                                      ['temperature of extremities'].transform('median'), inplace=True)

**6. "pain"**

    Боль определим на основе возраста и outcome. Видно, что чем выше возраст, тем выше уровень боли. Другие варианты дают много пропусков при группировке.

In [45]:
df_clean[['outcome', 'age', 'pain']].groupby(['outcome', 'age']).median()

pain
outcome age      
1.0     1     3.0
        9     3.0
2.0     1     4.0
        9     3.0
3.0     1     3.5
        9     2.0

In [46]:
df_clean['pain'].fillna(df_clean.groupby(['outcome', 'age'])\
                                      ['pain'].transform('median'), inplace=True)

In [47]:
(df_clean.isna().mean() * 100).round(2)

surgery?                      0.0
age                           0.0
rectal temperature            0.0
pulse                         0.0
respiratory rate              0.0
temperature of extremities    0.0
pain                          0.0
outcome                       0.0
dtype: float64

#### ПРИМЕЧАНИЕ
Домашнее задание сдается ссылкой [Google Colab](https://colab.research.google.com/).
Не сможем проверить или помочь, если вы пришлете:
- файлы;
- архивы;
- скриншоты кода.

Все обсуждения и консультации по выполнению домашнего задания ведутся только на соответствующем канале в slack.

##### Как правильно задавать вопросы аспирантам, преподавателям и коллегам
Прежде чем задать вопрос, попробуйте найти ответ в интернете. Навык самостоятельного поиска информации — один из важнейших. Каждый практикующий специалист любого уровня делает это ежедневно.

Сформулируйте вопрос по алгоритму:  
1) Что я делаю?  
2) Какого результата я ожидаю?  
3) Как фактический результат отличается от ожидаемого?  
4) Что я уже попробовал сделать, чтобы исправить проблему?  

По возможности прикрепите к вопросу скриншоты либо ссылки на код. Не выкладывайте все решение, оставляйте только проблемный и воспроизводимый участок кода.